In [2]:
import os
import re
import json
!pip install boto3
!pip install pickle
!pip install collections
!pip install unidecode
!pip install langdetect
import boto3
import pickle
from unidecode import unidecode
from collections import Counter
from langdetect import detect
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)
import numpy as np
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, DistilBertTokenizer
from transformers import DataCollatorWithPadding, PreTrainedTokenizerFast

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 7.7 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle
ERROR: Could not find a version that satisfies the requirement collections (from versions: none)
ERROR: No matching distribution found for collections
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 19.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=f7de66cf3e4118147044227f7b0ca30134c748987ba5d4cf66809da7bec7d5a2
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
S

In [3]:
from google.colab import drive
from google.colab import auth
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


**Cambiar rutas**

In [4]:
# Define the path
#model_path = "./path/to/model/artifacts/"
base_path="/content/drive/MyDrive/openalex-institution-parsing/"
artifacts_path = f"{base_path}Crudos/institution_tagger_v2_artifacts/"
model_path = f"{base_path}V2/002_Model/"


# Load the needed files
with open(os.path.join(artifacts_path, "departments_list1.pkl"), "rb") as f:
    departments_list = pickle.load(f)

print("Loaded list of departments")
print(departments_list)

with open(os.path.join(artifacts_path, "full_affiliation_dict1.pkl"), "rb") as f:
    full_affiliation_dict = pickle.load(f)

print("Loaded affiliation dictionary")
print(full_affiliation_dict)
print(len(full_affiliation_dict))


with open(os.path.join(artifacts_path, "multi_inst_names_ids1.pkl"), "rb") as f:
    multi_inst_names_ids = pickle.load(f)

print("Loaded list of institutions that have common name with other institutions.")

with open(os.path.join(artifacts_path, "countries_list_flat1.pkl"), "rb") as f:
    countries_list_flat = pickle.load(f)
print("Loaded flat list of countries")
print(countries_list_flat)

with open(os.path.join(artifacts_path, "countries.json"), "r") as f:
    countries_dict = json.load(f)

print("Loaded countries dictionary")
print(countries_dict
      )
with open(os.path.join(artifacts_path, "city_country_list1.pkl"), "rb") as f:
    city_country_list = pickle.load(f)

print("Loaded strings of city/country combinations")
print(city_country_list)

with open(os.path.join(artifacts_path, "affiliation_vocab_argentina.pkl"), "rb") as f:
    affiliation_vocab = pickle.load(f)

affiliation_vocab = {int(i):int(j) for i,j in affiliation_vocab.items()}

#se agrega la diccionario el valor -1

#affiliation_vocab[-1]=len(affiliation_vocab)

inverse_affiliation_vocab = {(i):(j) for j,i in affiliation_vocab.items()}

print("Loaded affiliation vocab")

##inverse_affiliation_vocab[-1]=len(inverse_affiliation_vocab)

print("affiliation_vocab_argentina")
print(affiliation_vocab)

print("inverse_affiliation_vocab")
print(inverse_affiliation_vocab)

Loaded list of departments
['Other departments', 'Materials Science and Engineering', 'Interdisciplinary Studies', 'Nursing & Midwifery', 'Nephrology', 'Internal Medicine', 'Biochemistry', 'Other Departments', 'Applied Science', 'Statistics', 'Electrical Engineering', 'Neuroscience', 'Computer Science', 'Electrical and Computer Engineering', 'Industrial & Systems Engineering', 'Communication', 'Political Science', 'Chemical and Biomolecular Engineering', 'Thoracic and Cardiovascular Surgery', 'Philosophy and Religious Studies', 'Agriculture', 'Engineering', 'English', 'Social Work', 'Surgery', 'Psychology', 'Applied Mechanics', 'Civil Engineering', 'Biological and Agricultural Engineering', 'Civil, Construction and Environmental Engineering', 'Foreign Languages and Literatures', 'Biology', 'Sociology and Anthropology', 'Medicine', 'Chemistry', 'Physics', 'Marine, Earth, and Atmospheric Sciences', 'Literature and Creative Writing', 'Information Technology', 'History', 'Biomedical Engine

In [5]:
len(full_affiliation_dict)

331

In [6]:
full_affiliation_dict

{3121987637: {'display_name': 'National University of the Littoral',
  'city': 'Santa Fe',
  'state': 'Santa Fe',
  'country': 'Argentina',
  'final_names': ['NationalUniversityoftheLittoral',
   'UniversidadNacionaldelLitoral',
   'UniversiteNationaleduLittoral'],
  'ror_id': '00pt8r998',
  'types': 'Education'},
 10108838: {'display_name': 'National University of Salta',
  'city': 'Salta',
  'state': 'Salta',
  'country': 'Argentina',
  'final_names': ['UniversidadNacionaldeSalta', 'NationalUniversityofSalta'],
  'ror_id': '00htwgm11',
  'types': 'Education'},
 168807105: {'display_name': 'National University of Luján',
  'city': 'Luján',
  'state': 'Buenos Aires',
  'country': 'Argentina',
  'final_names': ['UniversidadNacionaldeLujan', 'NationalUniversityofLujan'],
  'ror_id': '01yjy8p80',
  'types': 'Education'},
 11383257: {'display_name': 'Universidad del Salvador',
  'city': 'Buenos Aires',
  'state': 'Buenos Aires F.D.',
  'country': 'Argentina',
  'final_names': ['Universidad

In [7]:
# Load the tokenizers
language_tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased", return_tensors='tf')
data_collator = DataCollatorWithPadding(tokenizer=language_tokenizer,
                                        return_tensors='tf')

#basic_tokenizer = PreTrainedTokenizerFast(tokenizer_file=os.path.join(model_path, "basic_model_tokenizer"))
basic_tokenizer = PreTrainedTokenizerFast(tokenizer_file=os.path.join(model_path, "basic_model_tokenizer"))

# Load the models
#language_model = TFAutoModelForSequenceClassification.from_pretrained(os.path.join(model_path, "language_model"))
language_model = TFAutoModelForSequenceClassification.from_pretrained(os.path.join(model_path, "Result_model_lang"))
language_model.trainable = False

from tensorflow.keras.layers import TFSMLayer
basic_path=f'{base_path}V2/002_Model/Result_basic_model/'
##TFSMLayer Reload a Keras model/layer that was saved via SavedModel / ExportArchive.
basic_model = TFSMLayer(os.path.join(basic_path, "basic_model"), call_endpoint='serving_default')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Some layers from the model checkpoint at /content/drive/MyDrive/openalex-institution-parsing/V2/002_Model/Result_model_lang were not used when initializing TFDistilBertForSequenceClassification: ['dropout_39']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/openalex-institution-parsing/V2/002_Model/Result_model_lang and are newly initialized: ['dropout_19']
You should probably TRAIN this model on a down-stream

In [8]:
countries_list_flat[0:3]

['Denmark', 'Sierra Leone', 'Burundi']

In [9]:
def string_match_clean(text):
    #replace "&" with "and"
    if "r&d" not in text.lower():
        text = text.replace(" & ", " and ")

    # take country out
    if text.strip().endswith(")"):
        for country in countries_list_flat:
            if text.strip().endswith(f"({country})"):
                text = text.replace(f"({country})", "")

    # use unidecode
    text = unidecode(text.strip())

    # replacing common abbreviations
    text = text.replace("Univ.", "University")
    text = text.replace("Lab.", "Laboratory")
    text = text.replace("U.S. Army", "United States Army")
    text = text.replace("U.S. Navy", "United States Navy")
    text = text.replace("U.S. Air Force", "United States Air Force")

    # take out spaces, commas, dashes, periods, etcs
    text = re.sub("[^0-9a-zA-Z]", "", text)

    return text

def get_country_in_string(text):
    """
    Looks for countries in the affiliation string to be used in filtering later on.
    """
    countries_in_string = []
    _ = [countries_in_string.append(x) for x,y in countries_dict.items() if
         np.max([1 if re.search(fr"\b{i}\b", text) else 0 for i in y]) > 0]
    _ = [countries_in_string.append(x) for x,y in countries_dict.items() if
         np.max([1 if re.search(fr"\b{i}\b", text.replace(".","")) else 0 for i in y]) > 0]
    return list(set(countries_in_string))

def max_len_and_pad(tok_sent):
    """
    Processes the basic model data to the correct input length.
    """
    max_len = 128
    tok_sent = tok_sent[:max_len]
    tok_sent = tok_sent + [0]*(max_len - len(tok_sent))
    return tok_sent


def get_language(orig_aff_string):
    """
    Guesses the language of the affiliation string to be used for filtering later.
    """
    try:
        string_lang = detect(orig_aff_string)
    except:
        string_lang = 'en'

    return string_lang

def get_initial_pred(orig_aff_string, string_lang, countries_in_string, comma_split_len):
    """
    Initial hard-coded filtering of the affiliation text to ensure that meaningless strings
    and strings in other languages are not given an institution.
    """
    if string_lang in ['fa','ko','zh-cn','zh-tw','ja','uk','ru','vi','ar']:
        init_pred = None
    elif len(string_match_clean(str(orig_aff_string))) <=2:
        init_pred = None
    elif ((orig_aff_string.startswith("Dep") |
           orig_aff_string.startswith("School") |
           orig_aff_string.startswith("Ministry")) &
          (comma_split_len < 2) &
          (not countries_in_string)):
        init_pred = None
    elif orig_aff_string in departments_list:
        init_pred = None
    elif string_match_clean(str(orig_aff_string).strip()) in city_country_list:
        init_pred = None
    elif re.search(r"\b(LIANG|YANG|LIU|XIE|JIA|ZHANG)\b",
                   orig_aff_string):
        for inst_name in ["Hospital","University","School","Academy","Institute",
                          "Ministry","Laboratory","College"]:
            if inst_name in str(orig_aff_string):
                init_pred = 0
                break
            else:
                init_pred = None

    elif re.search(r"\b(et al)\b", orig_aff_string):
        if str(orig_aff_string).strip().endswith('et al'):
            init_pred = None
        else:
            init_pred = 0
    else:
        init_pred = 0
    return init_pred

def get_language_model_prediction(decoded_text, all_countries):
    """
    Preprocesses the decoded text and gets the output labels and scores for the language model.
    """
    lang_tok_data = language_tokenizer(decoded_text, truncation=True, padding=True, max_length=512)

    data = data_collator(lang_tok_data)
    all_scores, all_labels = tf.math.top_k(tf.nn.softmax(
            language_model.predict([data['input_ids'],
                                    data['attention_mask']]).logits).numpy(), 20)

    all_scores = all_scores.numpy().tolist()
    all_labels = all_labels.numpy().tolist()

    final_preds_scores = []
    for scores, labels, countries in zip(all_scores, all_labels, all_countries):
        final_pred, final_score, mapping = get_final_basic_or_language_model_pred(scores, labels, countries,
                                                                         affiliation_vocab,
                                                                         inverse_affiliation_vocab)
        final_preds_scores.append([final_pred, final_score, mapping])

    return final_preds_scores

def get_final_basic_or_language_model_pred(scores, labels, countries, vocab, inv_vocab):
    """
    Takes the scores and labels from either model and performs a quick country matching
    to see if the country found in the string can be matched to the country of the
    predicted institution.
    """
    mapped_labels = [inv_vocab[i] for i,j in zip(labels,scores) if i!=vocab[-1]]
    scores = [j for i,j in zip(labels,scores) if i!=vocab[-1]]
    final_pred = mapped_labels[0]
    final_score = scores[0]
    if not full_affiliation_dict[mapped_labels[0]]['country']:
        pass
    else:
        if not countries:
            pass
        else:
            for pred,score in zip(mapped_labels, scores):
                if not full_affiliation_dict[pred]['country']:
                    # trying pass instead of break to give time to find the correct country
                    pass
                elif full_affiliation_dict[pred]['country'] in countries:
                    final_pred = pred
                    final_score = score
                    break
                else:
                    pass
    return final_pred, final_score, mapped_labels

def get_similar_preds_to_remove(decoded_string, curr_preds):
    """
    Looks for organizations with similar/matching names and only predicts for one of those organizations.
    """
    preds_to_remove = []
    pred_display_names = [full_affiliation_dict[i]['display_name'] for i in curr_preds]
    counts_of_preds = Counter(pred_display_names)

    preds_array = np.array(curr_preds)
    preds_names_array = np.array(pred_display_names)

    for pred_name in counts_of_preds.items():
        temp_preds_to_remove = []
        to_use = []
        if pred_name[1] > 1:
            list_to_check = preds_array[preds_names_array == pred_name[0]].tolist()
            for pred in list_to_check:
                if string_match_clean(full_affiliation_dict[pred]['city']) in decoded_string:
                    to_use.append(pred)
                else:
                    temp_preds_to_remove.append(pred)
            if not to_use:
                to_use = temp_preds_to_remove[0]
                preds_to_remove += temp_preds_to_remove[1:]
            else:
                preds_to_remove += temp_preds_to_remove
        else:
            pass

    return preds_to_remove


def check_for_city_and_country_in_string(raw_sentence, countries, aff_dict_entry):
    """
    Checks for city and country and string for a common name institution.
    """
    if (aff_dict_entry['country'] in countries) & (aff_dict_entry['city'] in raw_sentence):
        return True
    else:
        return False

In [10]:
def get_basic_model_prediction(decoded_text, all_countries):
    """
    Preprocesses the decoded text and gets the output labels and scores for the basic model.
    """
    basic_tok_data = basic_tokenizer(decoded_text)['input_ids']
    basic_tok_data = [max_len_and_pad(x) for x in basic_tok_data]

    # Convert the padded data to a TensorFlow tensor
    #basic_tok_data=decoded_text['basic_tok_data'].tolist()
    basic_tok_tensor = tf.convert_to_tensor(basic_tok_data, dtype=tf.int64)## cambio

    # Proceed with the rest of your code
    # Get the prediction output from the model
    predictions = basic_model(basic_tok_tensor)

    # Extract the 'output_0' tensor from the predictions dictionary
    model_output = predictions['output_0']

    # Apply tf.math.top_k to the extracted tensor
    all_scores, all_labels = tf.math.top_k(model_output, 3)  # Use the extracted tensor here
    all_scores = all_scores.numpy().tolist()
    all_labels = all_labels.numpy().tolist()

    final_preds_scores = []
    for scores, labels, countries in zip(all_scores, all_labels, all_countries):
        final_pred, final_score, mapping = get_final_basic_or_language_model_pred(scores, labels, countries,
                                                                         affiliation_vocab,
                                                                         inverse_affiliation_vocab)
        final_preds_scores.append([final_pred, final_score, mapping])

    return final_preds_scores

In [11]:
def preprocess_affiliation_text(text):
        # Normalización de caracteres
        # Call the unidecode function within the unidecode module
        text = unidecode(text)
        # Convertir a minúsculas
        text = text.lower()
        # Remover caracteres especiales (si es necesario)
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
        return  text

In [12]:
def raw_data_to_predictions(df, lang_thresh, basic_thresh):
    """
    High level function to go from a raw input dataframe to the final dataframe with affiliation
    ID prediction.
    """
    # Implementing the functions above
    df['lang'] = df['affiliation_string'].apply(get_language)
    df['country_in_string'] = df['affiliation_string'].apply(get_country_in_string)
    df['comma_split_len'] = df['affiliation_string'].apply(lambda x: len([i if i else "" for i in
                                                                          x.split(",")]))

    # Gets initial indicator of whether or not the string should go through the models
    df['affiliation_id'] = df.apply(lambda x: get_initial_pred(x.affiliation_string, x.lang,
                                                               x.country_in_string, x.comma_split_len), axis=1)

    # Filter out strings that won't go through the models
    to_predict = df[df['affiliation_id']==0.0].drop_duplicates(subset=['affiliation_string']).copy()
    to_predict['affiliation_id'] = to_predict['affiliation_id'].astype('int')

    # Decode text so only ASCII characters are used
    to_predict['decoded_text'] = to_predict['affiliation_string'].apply(unidecode)

####agregado
      # Aplicar el preprocesamiento a los datos de entrenamiento, validación y prueba
    to_predict['decoded_text'] = to_predict['affiliation_string'].apply(preprocess_affiliation_text)
    print(to_predict.head(2))
    to_predict=pd.DataFrame(to_predict)
###final agregado


    # Get predictions and scores for each model
    to_predict['lang_pred_score'] = get_language_model_prediction(to_predict['decoded_text'].to_list(),
                                                                  to_predict['country_in_string'].to_list())
    to_predict['basic_pred_score'] = get_basic_model_prediction(to_predict['decoded_text'].to_list(),
                                                                to_predict['country_in_string'].to_list())

    # Get the final prediction for each affiliation string
    to_predict['affiliation_id'] = to_predict.apply(lambda x:
                                                    get_final_prediction(x.basic_pred_score,
                                                                         x.lang_pred_score,
                                                                         x.country_in_string,
                                                                         x.affiliation_string,
                                                                         lang_thresh, basic_thresh), axis=1)

    # Merge predictions to original dataframe to get the same order as the data that was requested
    final_df = df[['affiliation_string']].merge(to_predict[['affiliation_string','affiliation_id']],
                                                how='left', on='affiliation_string')

#     final_df['affiliation_id'] = final_df['affiliation_id'].fillna(-1).astype('int')
    return final_df

print("Models initialized")

Models initialized


In [13]:
def get_final_prediction(basic_pred_score, lang_pred_score, countries, raw_sentence, lang_thresh, basic_thresh):
    """
    Performs the model comparison and filtering to get the final prediction.
    """

    # Getting the individual preds and scores for both models
    pred_lang, score_lang, mapped_lang = lang_pred_score
    pred_basic, score_basic, mapped_basic = basic_pred_score

#     print(f"lang: {pred_lang} - {score_lang}")
#     print(f"basic: {pred_basic} - {score_basic}")

    # Logic for combining the two models

    final_preds = []
    final_scores = []
    final_cats = []
    check_pred = []
    if pred_lang == pred_basic:
        final_preds.append(pred_lang)
        final_scores.append(score_lang)
        final_cats.append('model_match')
        check_pred.append(pred_lang)
    elif score_basic > basic_thresh:
        final_preds.append(pred_basic)
        final_scores.append(score_basic)
        final_cats.append('basic_thresh')
        check_pred.append(pred_basic)
    elif score_lang > lang_thresh:
        final_preds.append(pred_lang)
        final_scores.append(score_lang)
        final_cats.append('lang_thresh')
        check_pred.append(pred_lang)
    elif (score_basic > 0.01) & ('China' in countries) & ('Natural Resource' in raw_sentence):
        final_preds.append(pred_basic)
        final_scores.append(score_basic)
        final_cats.append('basic_thresh_second')
        check_pred.append(pred_basic)
    else:
        final_preds.append(-1)
        final_scores.append(0.0)
        final_cats.append('nothing')

    print(final_preds)
    all_mapped = list(set(mapped_lang + mapped_basic))

    decoded_affiliation_string = string_match_clean(raw_sentence)
    all_mapped_strings = [full_affiliation_dict[i]['final_names'] for i in all_mapped]


    matched_preds = []
    matched_strings = []
#     print(f"RAW: {raw_sentence}")
#     print(f"CLEAN: {decoded_affiliation_string}")
#     print(f"COUNTRIES: {countries}")
    for inst_id, match_strings in zip(all_mapped, all_mapped_strings):
#         print(f"------{full_affiliation_dict[inst_id]['display_name']} - {inst_id}")
        if inst_id not in final_preds:
            for match_string in match_strings:
#                 print(f"------{match_string} ({full_affiliation_dict[inst_id]['display_name']} - {inst_id})")
                if match_string in decoded_affiliation_string:
#                     print("FOUND A MATCH")
                    if not full_affiliation_dict[inst_id]['country']:
#                         print("######match (no country_dict for aff ID)")
                        matched_preds.append(inst_id)
                        matched_strings.append(match_string)
                    elif not countries:
#                         print("######match (no country in string)")
                        if inst_id not in multi_inst_names_ids:
                            matched_preds.append(inst_id)
                            matched_strings.append(match_string)
                        else:
                            pass
                    elif full_affiliation_dict[inst_id]['country'] in countries:
#                         print("######match (country matches string)")
                        matched_preds.append(inst_id)
                        matched_strings.append(match_string)
                    else:
                        pass
                    break
                else:
                    pass
        else:
            pass

    # need to check for institutions that are a subset of another institution
    skip_matching = []
    for inst_id, matched_string in zip(matched_preds, matched_strings):
        for inst_id2, matched_string2 in zip(matched_preds, matched_strings):
            if (matched_string in matched_string2) & (matched_string != matched_string2):
                skip_matching.append(inst_id)

    if check_pred:
        for inst_id, matched_string in zip(matched_preds, matched_strings):
            for final_string in full_affiliation_dict[check_pred[0]]['final_names']:
                if matched_string in final_string:
                    skip_matching.append(inst_id)

    for matched_pred in matched_preds:
        if matched_pred not in skip_matching:
            final_preds.append(matched_pred)
            final_scores.append(0.95)
            final_cats.append('string_match')

    if (final_cats[0] == 'nothing') & (len(final_preds)>1):
        final_preds = final_preds[1:]
        final_scores = final_scores[1:]
        final_cats = final_cats[1:]

    # check if many names belong to same organization name (different locations)
    if (final_preds[0] != -1) & (len(final_preds)>1):
        final_display_names = [full_affiliation_dict[x]['display_name'] for x in final_preds]

        if len(final_display_names) == set(final_display_names):
            pass
        else:
            final_preds_after_removal = []
            final_scores_after_removal = []
            final_cats_after_removal = []
            preds_to_remove = get_similar_preds_to_remove(decoded_affiliation_string, final_preds)
            for temp_pred, temp_score, temp_cat in zip(final_preds, final_scores, final_cats):
                if temp_pred in preds_to_remove:
                    pass
                else:
                    final_preds_after_removal.append(temp_pred)
                    final_scores_after_removal.append(temp_score)
                    final_cats_after_removal.append(temp_cat)

            final_preds = final_preds_after_removal
            final_scores = final_scores_after_removal
            final_cats = final_cats_after_removal


    # check for multi-name institution problems (final check)
    preds_to_remove = []
    if final_preds[0] == -1:
        pass
    else:
        final_department_name_ids = [[x, str(full_affiliation_dict[x]['display_name'])] for x in final_preds if
                       (str(full_affiliation_dict[x]['display_name']).startswith("Department of") |
                        str(full_affiliation_dict[x]['display_name']).startswith("Department for"))]
        if final_department_name_ids:
            for temp_id in final_department_name_ids:
                if string_match_clean(temp_id[1]) not in string_match_clean(str(raw_sentence).strip()):
                    preds_to_remove.append(temp_id[0])
                elif not check_for_city_and_country_in_string(raw_sentence, countries,
                                                              full_affiliation_dict[temp_id[0]]):
                    preds_to_remove.append(temp_id[0])
                else:
                    pass


        if any(x in final_preds for x in multi_inst_names_ids):
            # go through logic
            if len(final_preds) == 1:
                pred_name = str(full_affiliation_dict[final_preds[0]]['display_name'])
                # check if it is exact string match
                if (string_match_clean(pred_name) == string_match_clean(str(raw_sentence).strip())):
                    final_preds = [-1]
                    final_scores = [0.0]
                    final_cats = ['nothing']
                elif pred_name.startswith("Department of"):
                    if ("College" in raw_sentence) or ("University" in raw_sentence):
                        final_preds = [-1]
                        final_scores = [0.0]
                        final_cats = ['nothing']
                    elif string_match_clean(pred_name) not in string_match_clean(str(raw_sentence).strip()):
                        final_preds = [-1]
                        final_scores = [0.0]
                        final_cats = ['nothing']

            else:
                non_multi_inst_name_preds = [x for x in final_preds if x not in multi_inst_names_ids]
                if len(non_multi_inst_name_preds) > 0:
                    for temp_pred, temp_score, temp_cat in zip(final_preds, final_scores, final_cats):
                        if temp_pred not in non_multi_inst_name_preds:
                            aff_dict_temp = full_affiliation_dict[temp_pred]
                            if aff_dict_temp['display_name'].startswith("Department of"):
                                if ("College" in raw_sentence) or ("University" in raw_sentence):
                                    preds_to_remove.append(temp_pred)
                                elif (string_match_clean(str(full_affiliation_dict[temp_pred]['display_name']))
                                      not in string_match_clean(str(raw_sentence).strip())):
                                    preds_to_remove.append(temp_pred)
                                else:
                                    if check_for_city_and_country_in_string(raw_sentence, countries, aff_dict_temp):
                                        pass
                                    else:
                                        preds_to_remove.append(temp_pred)
                            # check for city and country
                            elif aff_dict_temp['country'] in countries:
                                pass
                            else:
                                preds_to_remove.append(temp_pred)
                else:
                    pass
        else:
            pass

    true_final_preds = [x for x,y,z in zip(final_preds, final_scores, final_cats) if x not in preds_to_remove]
    true_final_scores = [y for x,y,z in zip(final_preds, final_scores, final_cats) if x not in preds_to_remove]
    true_final_cats = [z for x,y,z in zip(final_preds, final_scores, final_cats) if x not in preds_to_remove]

    if not true_final_preds:
        true_final_preds = [-1]
        true_final_scores = [0.0]
        true_final_cats = ['nothing']
    return [true_final_preds, true_final_scores, true_final_cats]

def raw_data_to_predictions(df, lang_thresh, basic_thresh):
    """
    High level function to go from a raw input dataframe to the final dataframe with affiliation
    ID prediction.
    """
    # Implementing the functions above
    df['lang'] = df['affiliation_string'].apply(get_language)
    df['country_in_string'] = df['affiliation_string'].apply(get_country_in_string)
    df['comma_split_len'] = df['affiliation_string'].apply(lambda x: len([i if i else "" for i in
                                                                          x.split(",")]))

    # Gets initial indicator of whether or not the string should go through the models
    df['affiliation_id'] = df.apply(lambda x: get_initial_pred(x.affiliation_string, x.lang,
                                                               x.country_in_string, x.comma_split_len), axis=1)

    # Filter out strings that won't go through the models
    to_predict = df[df['affiliation_id']==0.0].drop_duplicates(subset=['affiliation_string']).copy()
    to_predict['affiliation_id'] = to_predict['affiliation_id'].astype('int')

    # Decode text so only ASCII characters are used
    to_predict['decoded_text'] = to_predict['affiliation_string'].apply(unidecode)

    # Get predictions and scores for each model
    to_predict['lang_pred_score'] = get_language_model_prediction(to_predict['decoded_text'].to_list(),
                                                                  to_predict['country_in_string'].to_list())
    to_predict['basic_pred_score'] = get_basic_model_prediction(to_predict['decoded_text'].to_list(),
                                                                to_predict['country_in_string'].to_list())

    # Get the final prediction for each affiliation string
    to_predict['affiliation_id'] = to_predict.apply(lambda x:
                                                    get_final_prediction(x.basic_pred_score,
                                                                         x.lang_pred_score,
                                                                         x.country_in_string,
                                                                         x.affiliation_string,
                                                                         lang_thresh, basic_thresh), axis=1)

    # Merge predictions to original dataframe to get the same order as the data that was requested
    final_df = df[['affiliation_string']].merge(to_predict[['affiliation_string','affiliation_id']],
                                                how='left', on='affiliation_string')

#     final_df['affiliation_id'] = final_df['affiliation_id'].fillna(-1).astype('int')
    return final_df


print("Models initialized")

Models initialized


### Loading all gold data

In [14]:
def get_preds_display_names(all_preds):
    if isinstance(all_preds, float):
        return []
    elif isinstance(all_preds[0][0], int):
        if all_preds[0][0] == -1:
            return []
        else:
            return [f"{i} - {full_affiliation_dict.get(i).get('display_name')}"
                    if full_affiliation_dict.get(i) else "-1 - None" for i in all_preds[0]]
    else:
        return []

In [15]:
def get_labels_display_names(all_labels):
    if isinstance(all_labels, list):
        return [f"{i} - {full_affiliation_dict.get(i).get('display_name')}"
                    if full_affiliation_dict.get(i) else "-1 - None" for i in all_labels].copy()
    else:
        return []

In [16]:
def get_preds_all_or_original(all_preds, pred_type='all'):
    if isinstance(all_preds, float):
        return [-1]
    elif isinstance(all_preds[0][0], int):
        if pred_type=='all':
            return [i for i in all_preds[0]]
        else:
            final_preds = []

            for preds, scores, cats in zip(all_preds[0], all_preds[1], all_preds[2]):
                if cats == 'string_match':
                    pass
                else:
                    final_preds.append(preds)

            if not final_preds:
                final_preds = [-1]

            return final_preds
    else:
        return [-1]

Multiple different datasets were used for testing, refer to the documentation to find out more about them. The following code gathers all the datasets into a single dataframe so they can be run through the model.

In [17]:
multi_string = pd.read_csv(f"{base_path}Crudos/Datos de testeo/multi_string_inst_openalex.tsv", sep="\t") \
    [['paper_id','affiliation_string','labels','dataset']]
#print(multi_string.head(1))
multi_string['labels'] = multi_string['labels'].apply(lambda x: [int(i) for i in x.split("||||")])

cwts_1 = pd.read_csv(f"{base_path}Crudos/Datos de testeo/cwts_related_labeled.tsv", sep="\t") \
    [['paper_id','affiliation_string','labels','dataset']]
cwts_1['paper_id'] = cwts_1['paper_id'].apply(lambda x: int(x[1:]))
cwts_1['labels'] = cwts_1['labels'].apply(lambda x: [int(i.strip()) for i in x[1:-1].split(",")])

cwts_2 = pd.read_csv(f"{base_path}Crudos/Datos de testeo/cwts_no_relation_labeled.tsv", sep="\t") \
    [['paper_id','affiliation_string','labels','dataset']]
cwts_2['paper_id'] = cwts_2['paper_id'].apply(lambda x: int(x[1:]))
cwts_2['labels'] = cwts_2['labels'].apply(lambda x: [int(i.strip()) for i in x[1:-1].split(",")])

sampled_200 = pd.read_csv(f"{base_path}Crudos/Datos de testeo/sampled_200_labeled.tsv", sep="\t") \
    [['paper_id','affiliation_string','labels','dataset']]
sampled_200['labels'] = sampled_200['labels'].apply(lambda x: [int(i.strip()) for i in x[1:-1].split(",")])
sampled_200['dataset'] = "gold_random"

all_gold = pd.read_csv(f"{base_path}Crudos/Datos de testeo/gold_data_institution_parsing_labeled.tsv", sep="\t") \
    [['paper_id','affiliation_string','labels','dataset']]
all_gold['dataset'] = all_gold['dataset'].replace('gold_500','gold_hard').replace('gold_1000','gold_easy')
all_gold['labels'] = all_gold['labels'].apply(lambda x: [int(i.strip()) for i in x[1:-1].split(",")])



In [18]:
##opcion 1: con datos de aws
"""all_data = pd.concat([multi_string, cwts_1, cwts_2, sampled_200, all_gold], axis=0) \
    .drop_duplicates(subset=['affiliation_string'])
all_data.head()
#all_data['labels'] = all_data['labels'].apply(lambda x: [int(i.strip()) for i in x[1:-1].split(",")])
all_data=all_data.explode('labels')
all_data['labels'] = all_data['labels'].astype(int)
print(all_data.info())
all_data.head(3)
all_data=all_data[
         (all_data['labels']==3130805194)|
         (all_data['labels']==4210089990)|
         (all_data['labels']==9340077)|
         (all_data['labels']==232641801)|
         (all_data['labels']==15366983)|
         (all_data['labels']==4210122565)]
print(all_data.head(2))"""


'all_data = pd.concat([multi_string, cwts_1, cwts_2, sampled_200, all_gold], axis=0)     .drop_duplicates(subset=[\'affiliation_string\'])\nall_data.head()\n#all_data[\'labels\'] = all_data[\'labels\'].apply(lambda x: [int(i.strip()) for i in x[1:-1].split(",")])\nall_data=all_data.explode(\'labels\')\nall_data[\'labels\'] = all_data[\'labels\'].astype(int)\nprint(all_data.info())\nall_data.head(3)\nall_data=all_data[\n         (all_data[\'labels\']==3130805194)|\n         (all_data[\'labels\']==4210089990)|\n         (all_data[\'labels\']==9340077)|\n         (all_data[\'labels\']==232641801)|\n         (all_data[\'labels\']==15366983)|\n         (all_data[\'labels\']==4210122565)]\nprint(all_data.head(2))'

In [19]:
### opción 2: con muestra aleatoria creada a partir de openalex en notebook 1 (spark)
all_data = pd.read_csv(f"{base_path}V2/002_Model/affiliations_para_testeo.csv")
all_data=all_data.rename(columns={"paper_ids": "paper_id"})
all_data=all_data.explode('labels','affiliation_string')
all_data['labels'] = all_data['labels'].astype(int)
all_data['dataset']="muestra_authorship"
all_data.info()
all_data.shape

#all_data=all_data[all_data['affiliation_string'].str.contains('Arg')]
all_data.head()
#all_data.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          180 non-null    int64 
 1   labels              180 non-null    int64 
 2   affiliation_string  180 non-null    object
 3   paper_id            180 non-null    object
 4   dataset             180 non-null    object
dtypes: int64(2), object(3)
memory usage: 7.2+ KB


,Unnamed: 0,labels,affiliation_string,paper_id,dataset
0,7539,107639228,University of Notre Dame-Mendoza College of Business,W2137582495,muestra_authorship
1,6542,4210102755,"Centro Internacional de Métodos Computacionales en Ingenierı´a (CIMEC-INTEC), Universidad Nacional del Litoral (UNL), Consejo Nacional de Investigaciones Científicas y Técnicas (CONICET), Güemes 3450, S3000GLN, Santa Fe, Argentina",W2002813019,muestra_authorship
2,3023,3130805194,"Centro de Investigacion Veterinaria de Tandil, Facultad de Ciencias Veterinarias, Universidad Nacional del Centro de la Provincia de Buenos Aires, Tandil, Argentina",W3026900421,muestra_authorship
3,6898,151201029,"NCAR, P.O. Box 3000, Boulder, Colorado 80307-3000, USA",W1964422437,muestra_authorship
4,1430,189075073,"Institute of Materials Science and Technology (INTEMA), National Research Council-National University of Mar del Plata (CONICET-UNMdP), Juan B. Justo 4302, Mar del Plata, Argentina",W2052732316,muestra_authorship


In [20]:
all_data['labels'].value_counts()

,count
labels,
151201029,47
24354313,15
53241121,7
4141589,6
874386039,6
4210086432,5
4210134950,4
4210159438,4
70368782,4


In [ ]:
### Opción 3: con dataset creado para argentina (Santiago)
"""all_data = pd.read_csv(f"{base_path}V2/testeo_ar.tsv",sep="\t")
all_data=all_data.explode('labels')
all_data['labels'] = all_data['labels'].str.replace('[', '')
all_data['labels'] = all_data['labels'].str.replace(']', '')
all_data['labels'] = all_data['labels'].astype(str)

all_data['labels'] = all_data['labels'].astype(int)
print(all_data.info())
print(all_data.head(3))
all_data.shape"""


'all_data = pd.read_csv(f"{base_path}V2/testeo_ar.tsv",sep="\t")\nall_data=all_data.explode(\'labels\')\nall_data[\'labels\'] = all_data[\'labels\'].str.replace(\'[\', \'\')\nall_data[\'labels\'] = all_data[\'labels\'].str.replace(\']\', \'\')\nall_data[\'labels\'] = all_data[\'labels\'].astype(str)\n\nall_data[\'labels\'] = all_data[\'labels\'].astype(int)\nprint(all_data.info())\nprint(all_data.head(3))\nall_data.shape'

## Testing

In [ ]:
def get_confusion_matrix(labels, preds):
    TP=0 #true positive
    FP=0 #false positive
    TN=0 #true negative
    FN=0 #false negative
    if labels[0] == -1:
        if preds[0] != -1:
            FP = len(preds)
        else:
            TN = 1
    elif preds[0] == -1:
        FN = len(labels)
    else:
        TP = sum([1 for x in preds if x in labels]) #true positive
        FP = sum([1 for x in preds if x not in labels]) #false positive
        FN = sum([1 for x in labels if x not in preds]) #false neative

    return [TP, FP, TN, FN]

In [ ]:
def create_preview(aff_obj):
    basic_obj = aff_obj[0]
    lang_obj = aff_obj[1]

    basic_preds = [str(x) for x in basic_obj[0]]
    basic_scores = [round(x, 5) for x in basic_obj[1]]

    lang_preds = [str(x) for x in lang_obj[0]]
    lang_scores = [round(x, 5) for x in lang_obj[1]]

    basic_affs = [institutions.get(int(x)) for x in basic_preds]

    basic_ror = [x.get('ror_id') if x else "" for x in basic_affs]
    basic_aff_name = [x.get('display_name') if x else "" for x in basic_affs]
    basic_city_name = [x.get('city') if x else "" for x in basic_affs]
    basic_country_name = [x.get('country') if x else "" for x in basic_affs]

    lang_affs = [institutions.get(int(x)) for x in lang_preds]

    lang_ror = [x.get('ror_id') if x else "" for x in lang_affs]
    lang_aff_name = [x.get('display_name') if x else "" for x in lang_affs]
    lang_city_name = [x.get('city') if x else "" for x in lang_affs]
    lang_country_name = [x.get('country') if x else "" for x in lang_affs]

    preview_df = pd.DataFrame(zip(basic_preds, lang_preds, basic_ror, lang_ror,
                                  basic_aff_name, basic_city_name, basic_country_name,
                                  basic_scores,lang_aff_name, lang_city_name,
                                  lang_country_name, lang_scores),
                             columns=['basic_pred','lang_pred','basic_ror','lang_ror','basic_aff_name',
                                      'basic_city_name', 'basic_country_name', 'basic_score','lang_aff_name',
                                      'lang_city_name','lang_country_name', 'lang_score'])

    return preview_df

In [ ]:
def get_similar_preds_to_remove(decoded_string, curr_preds):
    """
    Looks for organizations with similar/matching names and only predicts for one of those organizations.
    """
    preds_to_remove = []
    pred_display_names = [full_affiliation_dict[i]['display_name'] for i in curr_preds]

    # Convert list to tuple or string if necessary
    # Ensure all elements are strings to create a homogeneous array
    pred_display_names = [str(x) for x in pred_display_names]

    counts_of_preds = Counter(pred_display_names)

    preds_array = np.array(curr_preds)
    preds_names_array = np.array(pred_display_names)

    for pred_name in counts_of_preds.items():
        temp_preds_to_remove = []
        to_use = []
        if pred_name[1] > 1:
            list_to_check = preds_array[preds_names_array == pred_name[0]].tolist()
            for pred in list_to_check:
                if string_match_clean(full_affiliation_dict[pred]['city']) in decoded_string:
                    to_use.append(pred)
                else:
                    temp_preds_to_remove.append(pred)
            if not to_use:
                to_use = temp_preds_to_remove[0]
                preds_to_remove += temp_preds_to_remove[1:]
            else:
                preds_to_remove += temp_preds_to_remove
        else:
            pass

    return preds_to_remove

In [ ]:
raw_data_to_predictions(all_data, lang_thresh=0.99, basic_thresh=0.99).head(4)

5/5 [==============================] - 3s 120ms/step
[100084298]
[-1]
[3130805194]
[24354313]
[189075073]
[-1]
[24354313]
[-1]
[4210137987]
[-1]
[-1]
[-1]
[4210157909]
[-1]
[874386039]
[24354313]
[189075073]
[151201029]
[4210097725]
[-1]
[-1]
[-1]
[4210110409]
[-1]
[-1]
[-1]
[24354313]
[-1]
[4210156734]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[4210120094]
[4210136553]
[4210159438]
[-1]
[3017690349]
[4210108200]
[4210159438]
[4210120094]
[-1]
[-1]
[-1]
[4141589]
[-1]
[-1]
[-1]
[44375435]
[-1]
[-1]
[4210117969]
[70368782]
[-1]
[-1]
[-1]
[-1]
[4210133962]
[-1]
[4210157909]
[-1]
[24354313]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[151201029]
[4141589]
[-1]
[39553462]
[-1]
[-1]
[-1]
[4210112829]
[874386039]
[-1]
[-1]
[29406707]
[4210120094]
[3129902936]
[4210133962]
[-1]
[-1]
[-1]
[-1]
[-1]
[4210156734]
[-1]
[-1]
[2801749607]
[4210136257]
[228664971]
[-1]
[189075073]
[297007282]
[-1]
[166401450]
[24354313]
[3130083209]
[4210147606]
[-1]
[4210108200]
[-1]
[-1]
[4210159438]

,affiliation_string,affiliation_id
0,University of Notre Dame-Mendoza College of Business,"[[100084298], [0.3748112618923187], [model_match]]"
1,"Centro Internacional de Métodos Computacionales en Ingenierı´a (CIMEC-INTEC), Universidad Nacional del Litoral (UNL), Consejo Nacional de Investigaciones Científicas y Técnicas (CONICET), Güemes 3450, S3000GLN, Santa Fe, Argentina","[[3121987637], [0.95], [string_match]]"
2,"Centro de Investigacion Veterinaria de Tandil, Facultad de Ciencias Veterinarias, Universidad Nacional del Centro de la Provincia de Buenos Aires, Tandil, Argentina","[[3130805194], [0.4731089770793915], [model_match]]"
3,"NCAR, P.O. Box 3000, Boulder, Colorado 80307-3000, USA","[[24354313], [0.9994606375694275], [basic_thresh]]"


In [ ]:
%%time
all_preds = raw_data_to_predictions(all_data, lang_thresh=0.99, basic_thresh=0.99)\
    .merge(all_data[['paper_id','affiliation_string','labels','dataset']])

all_preds1=all_preds

5/5 [==============================] - 1s 126ms/step
[100084298]
[-1]
[3130805194]
[24354313]
[189075073]
[-1]
[24354313]
[-1]
[4210137987]
[-1]
[-1]
[-1]
[4210157909]
[-1]
[874386039]
[24354313]
[189075073]
[151201029]
[4210097725]
[-1]
[-1]
[-1]
[4210110409]
[-1]
[-1]
[-1]
[24354313]
[-1]
[4210156734]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[4210120094]
[4210136553]
[4210159438]
[-1]
[3017690349]
[4210108200]
[4210159438]
[4210120094]
[-1]
[-1]
[-1]
[4141589]
[-1]
[-1]
[-1]
[44375435]
[-1]
[-1]
[4210117969]
[70368782]
[-1]
[-1]
[-1]
[-1]
[4210133962]
[-1]
[4210157909]
[-1]
[24354313]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[151201029]
[4141589]
[-1]
[39553462]
[-1]
[-1]
[-1]
[4210112829]
[874386039]
[-1]
[-1]
[29406707]
[4210120094]
[3129902936]
[4210133962]
[-1]
[-1]
[-1]
[-1]
[-1]
[4210156734]
[-1]
[-1]
[2801749607]
[4210136257]
[228664971]
[-1]
[189075073]
[297007282]
[-1]
[166401450]
[24354313]
[3130083209]
[4210147606]
[-1]
[4210108200]
[-1]
[-1]
[4210159438]

In [ ]:
all_preds1.head(2)

,affiliation_string,affiliation_id,paper_id,labels,dataset
0,University of Notre Dame-Mendoza College of Business,"[[100084298], [0.3748112618923187], [model_match]]",W2137582495,107639228,muestra_authorship
1,"Centro Internacional de Métodos Computacionales en Ingenierı´a (CIMEC-INTEC), Universidad Nacional del Litoral (UNL), Consejo Nacional de Investigaciones Científicas y Técnicas (CONICET), Güemes 3450, S3000GLN, Santa Fe, Argentina","[[3121987637], [0.95], [string_match]]",W2002813019,4210102755,muestra_authorship


In [ ]:
all_preds1['preds_name'] = all_preds1['affiliation_id'].apply(lambda x: get_preds_display_names(x))

all_preds1['preds_model_and_string_matching'] = all_preds1['affiliation_id'] \
    .apply(lambda x: get_preds_all_or_original(x, 'all'))

all_preds1['preds_model_only'] = all_preds1['affiliation_id']\
    .apply(lambda x: get_preds_all_or_original(x, 'model_only'))

all_preds1['preds_model_and_string_matching'] = all_preds1['preds_model_and_string_matching'] \
    .apply(lambda x: [int(i) if ~np.isnan(i) else -1 for i in x])

all_preds1['preds_model_only'] = all_preds1['preds_model_only']\
    .apply(lambda x: [int(i) if ~np.isnan(i) else -1 for i in x])

all_preds1.head(2)

,affiliation_string,affiliation_id,paper_id,labels,dataset,preds_name,preds_model_and_string_matching,preds_model_only
0,University of Notre Dame-Mendoza College of Business,"[[100084298], [0.3748112618923187], [model_match]]",W2137582495,107639228,muestra_authorship,[100084298 - University of Mendoza],[100084298],[100084298]
1,"Centro Internacional de Métodos Computacionales en Ingenierı´a (CIMEC-INTEC), Universidad Nacional del Litoral (UNL), Consejo Nacional de Investigaciones Científicas y Técnicas (CONICET), Güemes 3450, S3000GLN, Santa Fe, Argentina","[[3121987637], [0.95], [string_match]]",W2002813019,4210102755,muestra_authorship,[3121987637 - National University of the Littoral],[3121987637],[-1]


In [ ]:
all_preds1.info()
all_preds1['labels']=all_preds['labels'].astype(int)
all_preds1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 8 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   affiliation_string               250 non-null    object
 1   affiliation_id                   250 non-null    object
 2   paper_id                         250 non-null    object
 3   labels                           250 non-null    int64 
 4   dataset                          250 non-null    object
 5   preds_name                       250 non-null    object
 6   preds_model_and_string_matching  250 non-null    object
 7   preds_model_only                 250 non-null    object
dtypes: int64(1), object(7)
memory usage: 15.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 8 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   affi

In [ ]:
all_preds1['labels'].head(2)

,labels
0,107639228
1,4210102755


In [ ]:
def get_labels_display_names(all_labels):
    if isinstance(all_labels, list):
        return [f"{i} - {full_affiliation_dict.get(i).get('display_name')}"
                    if full_affiliation_dict.get(i) else "-1 - None" for i in all_labels].copy()
    else:
        return []

get_labels_display_names(all_preds1['labels'])

[]

In [ ]:
def get_confusion_matrix(labels, preds):
    """
    Calculates the confusion matrix for a single row of predictions.

    Args:
        labels: The true labels for the affiliation. This should be an iterable (list or array-like).
        preds: The predicted labels for the affiliation. This should be an iterable (list or array-like).

    Returns:
        A dictionary containing the confusion matrix values (TP, FP, TN, FN).
    """
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    # Check if labels is a single value and convert to a list if needed
    if not isinstance(labels, (list, tuple, np.ndarray)):  # Check if it's not already iterable
        labels = [labels]  # Wrap single integer in a list
    if not isinstance(preds, (list, tuple, np.ndarray)):  # Check if it's not already iterable
        preds = [preds]  # Wrap single integer in a list


    if labels[0] == -1:
        if preds[0] != -1:
            FP = len(preds)
        else:
            TN = 1
    else:
        # Check for true positives: label is not -1 and prediction matches any label
        TP = sum(1 for pred in preds if pred in labels and pred != -1)
        # Check for false positives: prediction is not -1 and not in true labels
        FP = sum(1 for pred in preds if pred != -1 and pred not in labels)
        FN = len(labels) - TP  # False negatives: true labels missed

    return {'TP': TP, 'FP': FP, 'TN': TN, 'FN': FN}

In [ ]:
##paso full_affiliation_dict a dataframe para matchear label de all_preds con display name de full_affiliation_dict
a=pd.DataFrame.from_dict(full_affiliation_dict,orient='index')
a=a[['display_name']]
a.index.name = 'labels'
print(a.head(10))
a.shape


                                         display_name
labels                                               
3121987637        National University of the Littoral
10108838                 National University of Salta
168807105                National University of Luján
11383257                     Universidad del Salvador
187398979                         Favaloro University
4210099959             Hospital Universitario Austral
73822459              National University of San Luis
15366983            National University of Río Cuarto
70368782               National University of Comahue
4210119967  Hospital de Clínicas "José de San Martín"


(331, 1)

In [ ]:

a=all_preds1.merge(a,on='labels').head()
all_preds1['labels_name'] = a['display_name']
all_preds1.head(2)



,affiliation_string,affiliation_id,paper_id,labels,dataset,preds_name,preds_model_and_string_matching,preds_model_only,labels_name,conf_mat_model_and_string_matching,conf_mat_model_only,has_FP,has_FN,TP,FP,TN,FN
0,University of Notre Dame-Mendoza College of Business,"[[100084298], [0.3748112618923187], [model_match]]",W2137582495,107639228,muestra_authorship,[100084298 - University of Mendoza],[100084298],[100084298],Instituto de Desarrollo Tecnológico para la Industria Química,"{'TP': 0, 'FP': 1, 'TN': 0, 'FN': 1}","{'TP': 0, 'FP': 1, 'TN': 0, 'FN': 1}",1,1,0,1,0,1
1,"Centro Internacional de Métodos Computacionales en Ingenierı´a (CIMEC-INTEC), Universidad Nacional del Litoral (UNL), Consejo Nacional de Investigaciones Científicas y Técnicas (CONICET), Güemes 3450, S3000GLN, Santa Fe, Argentina","[[3121987637], [0.95], [string_match]]",W2002813019,4210102755,muestra_authorship,[3121987637 - National University of the Littoral],[3121987637],[-1],Consejo Nacional de Investigaciones Científicas y Técnicas,"{'TP': 0, 'FP': 1, 'TN': 0, 'FN': 1}","{'TP': 0, 'FP': 0, 'TN': 0, 'FN': 1}",1,1,0,1,0,1


In [ ]:
all_preds1['conf_mat_model_and_string_matching'] = all_preds1\
    .apply(lambda x: get_confusion_matrix(x.labels, x.preds_model_and_string_matching), axis=1)
all_preds1['conf_mat_model_only'] = all_preds1.apply(lambda x: get_confusion_matrix(x.labels,
                                                                                  x.preds_model_only), axis=1)
all_preds1.head(2)

,affiliation_string,affiliation_id,paper_id,labels,dataset,preds_name,preds_model_and_string_matching,preds_model_only,labels_name,conf_mat_model_and_string_matching,conf_mat_model_only,has_FP,has_FN,TP,FP,TN,FN
0,University of Notre Dame-Mendoza College of Business,"[[100084298], [0.3748112618923187], [model_match]]",W2137582495,107639228,muestra_authorship,[100084298 - University of Mendoza],[100084298],[100084298],Instituto de Desarrollo Tecnológico para la Industria Química,"{'TP': 0, 'FP': 1, 'TN': 0, 'FN': 1}","{'TP': 0, 'FP': 1, 'TN': 0, 'FN': 1}",1,1,0,1,0,1
1,"Centro Internacional de Métodos Computacionales en Ingenierı´a (CIMEC-INTEC), Universidad Nacional del Litoral (UNL), Consejo Nacional de Investigaciones Científicas y Técnicas (CONICET), Güemes 3450, S3000GLN, Santa Fe, Argentina","[[3121987637], [0.95], [string_match]]",W2002813019,4210102755,muestra_authorship,[3121987637 - National University of the Littoral],[3121987637],[-1],Consejo Nacional de Investigaciones Científicas y Técnicas,"{'TP': 0, 'FP': 1, 'TN': 0, 'FN': 1}","{'TP': 0, 'FP': 0, 'TN': 0, 'FN': 1}",1,1,0,1,0,1


In [ ]:
all_preds1['has_FP'] = all_preds1['conf_mat_model_and_string_matching'].apply(lambda x: 1 if x['FP'] > 0 else 0)
all_preds1['has_FN'] = all_preds1['conf_mat_model_and_string_matching'].apply(lambda x: 1 if x['FN'] > 0 else 0)

all_preds1['TP'] = all_preds1['conf_mat_model_and_string_matching'].apply(lambda x: x['TP'])
all_preds1['FP'] = all_preds1['conf_mat_model_and_string_matching'].apply(lambda x: x['FP'])
all_preds1['TN'] = all_preds1['conf_mat_model_and_string_matching'].apply(lambda x: x['TN'])
all_preds1['FN'] = all_preds1['conf_mat_model_and_string_matching'].apply(lambda x: x['FN'])

all_preds.head(2)

,affiliation_string,affiliation_id,paper_id,labels,dataset,preds_name,preds_model_and_string_matching,preds_model_only,labels_name,conf_mat_model_and_string_matching,conf_mat_model_only,has_FP,has_FN,TP,FP,TN,FN
0,University of Notre Dame-Mendoza College of Business,"[[100084298], [0.3748112618923187], [model_match]]",W2137582495,107639228,muestra_authorship,[100084298 - University of Mendoza],[100084298],[100084298],Instituto de Desarrollo Tecnológico para la Industria Química,"{'TP': 0, 'FP': 1, 'TN': 0, 'FN': 1}","{'TP': 0, 'FP': 1, 'TN': 0, 'FN': 1}",1,1,0,1,0,1
1,"Centro Internacional de Métodos Computacionales en Ingenierı´a (CIMEC-INTEC), Universidad Nacional del Litoral (UNL), Consejo Nacional de Investigaciones Científicas y Técnicas (CONICET), Güemes 3450, S3000GLN, Santa Fe, Argentina","[[3121987637], [0.95], [string_match]]",W2002813019,4210102755,muestra_authorship,[3121987637 - National University of the Littoral],[3121987637],[-1],Consejo Nacional de Investigaciones Científicas y Técnicas,"{'TP': 0, 'FP': 1, 'TN': 0, 'FN': 1}","{'TP': 0, 'FP': 0, 'TN': 0, 'FN': 1}",1,1,0,1,0,1


### Overall Performance

As mentioned above, the folowing code generates the precision and recall for the data. Predictions are split up between ones that are made by the model (model_only) and predictions that are added on using smart string-matching (model_and_string_matching).

In [ ]:
model_and_string_matching_confs_1000 = all_preds1['conf_mat_model_and_string_matching'].tolist()
model_only_confs_1000 = all_preds1['conf_mat_model_only'].tolist()

print("--------- MODEL WITH STRING MATCHING ---------")
# Access dictionary values using keys instead of indices
print("Precision: ", round(sum([x['TP'] for x in model_and_string_matching_confs_1000])/
            (sum([x['TP'] for x in model_and_string_matching_confs_1000]) +
             sum([x['FP'] for x in model_and_string_matching_confs_1000])), 3))

print("Recall: ", round(sum([x['TP'] for x in model_and_string_matching_confs_1000])/
            (sum([x['TP'] for x in model_and_string_matching_confs_1000]) +
             sum([x['FN'] for x in model_and_string_matching_confs_1000])), 3))
print("")
print("--------- MODEL ONLY ---------")

# Access dictionary values using keys instead of indices
print("Precision: ", round(sum([x['TP'] for x in model_only_confs_1000])/
            (sum([x['TP'] for x in model_only_confs_1000]) + sum([x['FP'] for x in model_only_confs_1000])), 3))

print("Recall: ", round(sum([x['TP'] for x in model_only_confs_1000])/
            (sum([x['TP'] for x in model_only_confs_1000]) + sum([x['FN'] for x in model_only_confs_1000])), 3))

--------- MODEL WITH STRING MATCHING ---------
Precision:  0.369
Recall:  0.316

--------- MODEL ONLY ---------
Precision:  0.287
Recall:  0.1


In [ ]:
all_data['country_in_string'].value_counts()

,count
country_in_string,
[Argentina],129
[],19
[United States],7
[France],3
"[Argentina, United States]",3
[Switzerland],2
"[Argentina, Bolivia]",2
[Germany],2
[Spain],2


In [ ]:
!pip install colab-convert
!colab-convert /content/drive/MyDrive/openalex-institution-parsing/V2/002_Model/Notebooks/003_model_testing.ipynb /content/drive/MyDrive/openalex-institution-parsing/V2/002_Model/Notebooks/003_model_testing.py

  Preparing metadata (setup.py) ... done
  Created wheel for colab-convert: filename=colab_convert-2.0.5-py3-none-any.whl size=19207 sha256=17fa5f2eaafe27cd93aca9d64971c62bb316082ea580c4c05ada27409d44a050
  Stored in directory: /root/.cache/pip/wheels/a7/9f/0a/70f4be5eeba4a3fca9d7bcf68d5a4e97edb9f22be449cc1e8c
Successfully built colab-convert

---------------------------
----------[START]----------
---------------------------
[OK]   en_US Detected as a supported Language
[OK]   convert magic commands
[OK]   commenting out unsupported magic commands
[OK]   keeping new imports made by converter
[NOT]  showing outputs from converter

convert /content/drive/MyDrive/openalex-institution-parsing/V2/002_Model/Notebooks/003_model_testing.ipynb to /content/drive/MyDrive/openalex-institution-parsing/V2/002_Model/Notebooks/003_model_testing.py
finished
conversion took 0.5269 seconds

log file created:
/content/cc-outputs.log
